In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
#dataset = '../Dataset/Car/car'
#dataset = '../Dataset/Balance-sheet/train.txt'
#dataset = '../Dataset/Hayes-Roth/Hayes-Roth'
dataset = '../Dataset/Heart-Data/train.txt'
#dataset = '../Dataset/TicTacToe/tic tac toe'
#dataset = '../Dataset/adult.data'
#dataset = '../Dataset/mush.data'
#dataset = '../Dataset/nursery.data'
DATASET_SPLIT_RATIO = .1
np.random.seed(2)

In [3]:
def clasification_report(y_true, y_pred, output_dict):
    unique_labels = list(np.unique(y_true))
    ret  = {}
    ret['precision'] = 0
    ret['recall'] = 0
    ret['f1-measure'] = 0
    div = 0
    for label in unique_labels:
        TP = np.sum((np.array(y_true) == label) & (np.array(y_true) == np.array(y_pred)))
        TN = np.sum((np.array(y_true) != label) & (np.array(y_true) == np.array(y_pred)))
        FP = np.sum((np.array(y_true) == label) & (np.array(y_true) != np.array(y_pred)))
        FN = np.sum((np.array(y_true) != label) & (np.array(y_true) != np.array(y_pred)))
        ret['precision'] += TP / (TP + FP) * (np.sum((np.array(y_true) == label)) / len(y_true))
        ret['recall'] += TP / (TP+FN) * (np.sum((np.array(y_true) == label)) / len(y_true))
        ret['f1-measure'] += 2 * ((ret['precision'] * ret['recall']) / (ret['precision'] + ret['recall'])) * (np.sum((np.array(y_true) == label)) / len(y_true))
        div +=  (np.sum((np.array(y_true) == label)) / len(y_true))
    ret['precision'] /= div
    ret['recall'] /= div
    ret['f1-measure'] /= div
    return ret

In [4]:
class DecisionTreeClassifier:
    def __init__(self):
        self.number_of_nodes = 1
        self.root_node = 0
        self.node_identity = [-1]
        self.node_edge = [-1]
        self.node_edge_dict = [-1]
        self.mx_identity= [-1]

    def select_feature(self, data):
        list_of_features = list(data)
        list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
        label_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
        total_number_of_rows = len(data)
        entropy = 0
        for i in range(0, len(list_of_labels)):
            entropy = entropy + (- label_count[i]/total_number_of_rows) * (math.log2(label_count[i]/total_number_of_rows))
            # Total numer of rows in the entire dataset or the number of rows in the current node?
        #print('Entropy:', entropy)

        mx_gain_ratio = -1
        mx_idx = -1

        for i in range(0, len(list_of_features) - 1):
            # calculate information gain of i-th feature using the labels on data[list_of_features[-1]]
            list_of_possible_values = np.unique(data[list_of_features[i]], return_counts=True)[0]
            count_values = np.unique(data[list_of_features[i]], return_counts=True)[1]
            total_values = sum(count_values)
            information = 0
            split_info = 0
            #print(list_of_possible_values)
            #print(count_values)
            for j in range(len(list_of_possible_values)):
                information_label = 0
                for k in range(len(list_of_labels)):
                    denominator = len(data[(data[list_of_features[i]] == list_of_possible_values[j]) & (data[list(data)[-1]] == list_of_labels[k])])          
                    if denominator == 0:
                        continue
                    information_label = information_label + (- denominator / count_values[j]) * math.log2(denominator / count_values[j])
                information = information + information_label * (count_values[j]/total_values)
                split_info = split_info +  -(count_values[j]/total_values) * math.log2(count_values[j]/total_values)

            information_gain = entropy - information
            #split_info = 1
            gain_ratio = information_gain / split_info
            #print('Feature', i, ':', information_gain, split_info, gain_ratio)
            if gain_ratio > mx_gain_ratio:
                mx_gain_ratio = gain_ratio
                mx_idx = i

            #print('Feature', i, ':', information)
            #print('Feature', i, ':', information_gain)

        return mx_idx
    def train(self, node, data):
        list_of_features = list(data)
        list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
        list_of_labels_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
        if len(list_of_labels) == 1:
            self.node_identity[node] = list_of_labels[0]
            return
        majority_idx = np.argmax(list_of_labels_count)
        self.mx_identity[node] = list_of_labels[majority_idx]
        if len(list(data)) == 1:
            majority_idx = np.argmax(list_of_labels_count)
            self.node_identity[node] = list_of_labels[majority_idx] 
            return

        mx_idx = self.select_feature(data)
        #print(node, mx_idx)
        remaining_features = list_of_features[0:mx_idx] + list_of_features[mx_idx+1: ]
        #print('Selected feature:', mx_idx)
        #print('Remaining_features:', remaining_features)
        list_of_possible_values = np.unique(data[list_of_features[mx_idx]])
        self.node_edge[node] = list_of_features[mx_idx]
        self.node_edge_dict[node] = {}    
        for label in list_of_possible_values:
            new_data_frame = data[data[list_of_features[mx_idx]] == label][remaining_features]
            if len(new_data_frame) == 0:
                continue
            new_node = self.number_of_nodes
            self.node_edge_dict[node][label] = new_node
            self.node_identity.append(-1)
            self.node_edge.append(-1)
            self.node_edge_dict.append(-1)
            self.mx_identity.append(-1)
            self.number_of_nodes += 1
            self.train(new_node, new_data_frame)
        return 
    def predict(self, data):
        cur_node = 0
        while self.node_identity[cur_node] == -1:
            try:
                cur_node = self.node_edge_dict[cur_node][data[self.node_edge[cur_node]].values[0]]
            except:
                return self.mx_identity[cur_node]
        return self.node_identity[cur_node]
    def test(self, test):
        right_predictions = 0
        y_true = []
        y_pred = []
        for i in range(len(test)):
            y_pred.append(self.predict(test[i:i+1]))
            y_true.append(test[i:i+1][list(test)[-1]].values[0])
            if self.predict(test[i:i+1]) == test[i:i+1][list(test)[-1]].values[0]:
                right_predictions+=1
        ret_dict = classification_report(y_true, y_pred, output_dict=True)
        ret_dict['accuracy'] = round(right_predictions/len(test) * 100, 2)
        return ret_dict

In [5]:
def cross_validation(dataframe):
    important_classifier = DecisionTreeClassifier()
    accuracies = []
    data_in_a_single_fold = int(np.ceil(max(10, len(dataframe)/10)))
    #data_in_a_single_fold = int(np.ceil(.1*len(dataframe)))
    number_of_folds = int(np.ceil(len(dataframe)/data_in_a_single_fold))
    print('Number of folds:', number_of_folds)
    print('Number of rows in a single fold:', data_in_a_single_fold)
    for i in range(number_of_folds):
        #print(min((i)*data_in_a_single_fold, len(dataframe)), 'bal')
        a = dataframe.iloc[0 : min((i)*data_in_a_single_fold, len(dataframe))]

        #print('min', min((i+1) * data_in_a_single_fold, len(dataframe)))
        b = dataframe.iloc[min((i+1) * data_in_a_single_fold, len(dataframe)) :  len(dataframe)]
        #print(b.head())
        train = pd.concat([a, b]).reset_index().drop(columns = 'index')
        b.reset_index().drop(columns = 'index')
        
        test = dataframe.iloc[i * data_in_a_single_fold : min((i+1)*data_in_a_single_fold, len(dataframe))].reset_index().drop(columns = 'index')
        print('Number of training samples:', len(train))
        print('Number of test samples:', len(test))
        print('Labels:', np.unique(train[list(dataframe)[-1]]))
        important_classifier.train(0, train)    
        accuracies.append(important_classifier.test(test))
    return accuracies

In [6]:
dataframe = pd.read_csv(dataset, header=None)
dataframe = dataframe.sample(frac=1)
accuracies = cross_validation(dataframe)
#print('Accuracies:', accuracies)
#print('Average accuracy:', round(np.mean(accuracies), 2))

Number of folds: 10
Number of rows in a single fold: 19
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]


/home/immuntasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/home/immuntasir/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 168
Number of test samples: 19
Labels: [0 1]
Number of training samples: 171
Number of test samples: 16
Labels: [0 1]


In [7]:
avg_wei = {'recall':0, 'f1-score':0, 'precision':0, 'support':0}
avg_acc = 0
n = 0
for accu in accuracies:
    for key in avg_wei.keys():
        avg_wei[key] += accu['weighted avg'][key]
    avg_acc += accu['accuracy']
    n += 1
    print(accu['accuracy'])
    
for key in avg_wei.keys():
    avg_wei[key] /= n
    avg_wei[key] = round(avg_wei[key], 2)
avg_acc /= n
avg_acc = round(avg_acc, 2)

89.47
84.21
89.47
78.95
100.0
78.95
94.74
94.74
94.74
100.0


In [9]:
print(avg_acc)
print(avg_wei['precision'])
print(avg_wei['recall'])
print(avg_wei['f1-score'])



90.53
0.94
0.91
0.92


Car Dataset: 93.06%
Balance Sheet: 26.98%
Hayes-Roth: 50.0%
Heart Data: 89.47%
TicTacToe: 85.42%